In [2]:
import pandas as pd
import numpy as np
import csv
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import xgboost as xgb
import scipy
from geopy import distance
import geopy
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn import preprocessing


%matplotlib inline
sns.set()
plt.rcParams['figure.figsize'] = [16, 10]

#### Read Data

In [ ]:
PATH_TRAIN_DATASET = './data/train.csv'
PATH_TEST_DATASET = './data/test.csv'
PATH_SAMPLE_SUMBISSION = './data/sample_submission.csv'

In [ ]:
df_test = pd.read_csv(PATH_TEST_DATASET, infer_datetime_format=True, parse_dates=['pickup_datetime'],  index_col='id')
df_train = pd.read_csv(PATH_TRAIN_DATASET, infer_datetime_format=True,parse_dates=['pickup_datetime'], index_col='id')
df_sample_submission = pd.read_csv(PATH_SAMPLE_SUMBISSION)
df_original_train = df_train.copy()
df_original_test = df_test.copy()

#### A brief description of the data

As mentioned in the challenge's page, the provided training dataset contains the following fields, as can be verified bellow:

- `id` - a unique identifier for each trip
- `vendor_id` - a code indicating the provider associated with the trip record
- `pickup_datetime` - date and time when the meter was engaged
- `dropoff_datetime` - date and time when the meter was disengaged
- `passenger_count` - the number of passengers in the vehicle (driver entered value)
- `pickup_longitude` - the longitude where the meter was engaged
- `pickup_latitude` - the latitude where the meter was engaged
- `dropoff_longitude` - the longitude where the meter was disengaged
- `dropoff_latitude` - the latitude where the meter was disengaged
- `store_and_fwd_flag` - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server -`Y`=store and forward; `N`=not a store and forward trip
- `trip_duration` - duration of the trip in seconds

In [ ]:
df_train.columns

In [ ]:
df_train.head(5)

#### No Null Columns

It's important to notice that the provided database has been already preprocessed and cleaned, so no null values can be found in the base.

In [ ]:
df_train.isnull().sum()

## Analyzing the whole Dataset

#### Prepare the data for analysis

Before the analysis, a few steps must be done to prepare the data. Also, the dropoff_datetime column will be dropped because it's redundant since we have the trip_duration.

In [ ]:
df_train['pickup_datetime'] = df_train['pickup_datetime'].dt.to_pydatetime()
df_test['pickup_datetime'] = df_test['pickup_datetime'].dt.to_pydatetime()
df_train.drop('dropoff_datetime', axis=1, inplace=True)

#### Trip Duration

Since trip duration is the target variable, it will be the first to be checked. After aplying the describe function, a few strange values appear, like the min and max values, 1s and 3526282s (almost 980h) respectively. Trips with a such a low or high values for duration can decrease the accuracy of our model.

In [ ]:
df_train['trip_duration'].describe()

To get rid of the outliers, lets apply the the [Interquartile Range](https://en.wikipedia.org/wiki/Interquartile_range) technique

In [ ]:
Q1 = df_train['trip_duration'].quantile(0.25)
Q3 = df_train['trip_duration'].quantile(0.75)
IQR = Q3 - Q1
df_train = df_train[~((df_train['trip_duration'] < (Q1 - 1.5 * IQR)) |(df_train['trip_duration'] > (Q3 + 1.5 * IQR)))]
df_train['trip_duration'].describe()

Even applying the IQR technique, the minimum value for a trip duration keeps very low, so values bellow 90 seconds will also be disconsidered.

In [ ]:
df_train = df_train[df_train['trip_duration'] > 1]
df_train = df_train[df_train['trip_duration'] < 7200]

In [ ]:
sns.distplot(df_train['trip_duration'], axlabel='Trip Duration')

#### Analyzing the Lat-Long Columns

To understand the distribution we must plot the values for each of the Latitude and Longitude provided. From the graph is possible to see that there are some datapoints from trips starting and finishing outside NYC.

In [ ]:
def plot_lat_long(df_data):
    fig, ax = plt.subplots(2,2,figsize=(16, 10), sharex=False, sharey = False)
    sns.distplot(df_data['pickup_latitude'], axlabel = 'Pickup Latitude',ax=ax[0,0])
    sns.distplot(df_data['pickup_longitude'], axlabel = 'Pickup_Longitude', ax=ax[0,1])
    sns.distplot(df_data['dropoff_latitude'], axlabel = 'Dropoff Latitude', ax=ax[1, 0])
    sns.distplot(df_data['dropoff_longitude'], axlabel = 'Dropoff Longitude', ax=ax[1, 1])
    plt.show()
plot_lat_long(df_train)

#### Drop Values Starting or Finishing out of New York City

Select values only within the NYC bounding Box

In [ ]:
NYC_BOUNDING_BOX = [(40.4774,-74.2589), ( 40.9176, -73.7004)]

filter_lat_long = df_train['pickup_latitude'] < NYC_BOUNDING_BOX[1][0]
filter_lat_long &= df_train['pickup_latitude'] > NYC_BOUNDING_BOX[0][0]
filter_lat_long &= df_train['pickup_longitude'] < NYC_BOUNDING_BOX[1][1]
filter_lat_long &= df_train['pickup_longitude'] > NYC_BOUNDING_BOX[0][1]

filter_lat_long &= df_train['dropoff_latitude'] < NYC_BOUNDING_BOX[1][0]
filter_lat_long &= df_train['dropoff_latitude'] > NYC_BOUNDING_BOX[0][0]
filter_lat_long &= df_train['dropoff_longitude'] < NYC_BOUNDING_BOX[1][1]
filter_lat_long &= df_train['dropoff_longitude'] > NYC_BOUNDING_BOX[0][1]

df_train = df_train[filter_lat_long]

plot_lat_long(df_train)

#### Extracting New Features: DIstance and Average Speed

Two more features will be added using the fields provided. Since we have the coorinates from the pickup and dropoff points, we can calculate the distance between points. The distance function chosen for that is the Manhattan Distance, City Block distance or [Taxicab geometry](https://en.wikipedia.org/wiki/Taxicab_geometry). The final distance is given in Kilometers

The other feature is the average speed for the whole route, for that we will use the distance obtained in the previous step and the trip duration. The average speed will be in Kilometer per hour


In [ ]:
NYC_LAT_KILOMETER_PER_DEGREE = geopy.distance.distance((40.7831, -73.9712),(41.7831, -73.9712)).kilometers
NYC_LON_KILOMETER_PER_DEGREE = geopy.distance.distance((40.7831, -73.9712),(40.7831, -72.9712)).kilometers

NYC_DEGREE_KM = 111.05938787411571

def calculate_city_block_distance(df_data):
    delta_lat = np.absolute(df_data.pickup_latitude - df_data.dropoff_latitude) * NYC_LAT_KILOMETER_PER_DEGREE    
    delta_lon = np.absolute(df_data.pickup_longitude - df_data.dropoff_longitude) * NYC_LON_KILOMETER_PER_DEGREE    
    return delta_lat + delta_lon


df_train['distance'] = calculate_city_block_distance(df_train)
df_train['avg_speed'] = df_train['distance']/(df_train['trip_duration']/3600)

### Analyzing Distance and Average Speed


In [ ]:
fig, ax = plt.subplots(1,2,figsize=(16, 10), sharex=False, sharey = False)
sns.distplot(df_train['distance'], axlabel = 'Distance',ax=ax[0])
sns.distplot(df_train['avg_speed'], axlabel = 'Average Speed',ax=ax[1])

Even filtering small values for trip duration, we still can find some values that must be removed, since it don't reflects the reality, as can be seen in the graph above, where average speeds of 800 Km/h can be seen. To remove more sporious rows, we'll remove the any value with average speeds greater than 120 Km/h.
Also distances close to 0 can be found and must be removed, for that we'll remove any values smaller than 0.250km

Even filtering the values, is possible to see that the distance distribution is skewed, for this case, we will later apply a logarithmic transformation.

In [ ]:
df_train = df_train[df_train['avg_speed'] < 100]
df_train = df_train[df_train['avg_speed'] > 1]
df_train = df_train[df_train['distance'] > .25]

#### Analyzing Pickup Date

As discussed before, only the pickup date will be used. A few features were created from the pickup datetime, they are justified by the typical behavior that we expect to see in the data, like "Rush Hours" or less traffic at weekends and Holidays. 

To analyze that behavior the following features were created: weekdays, holidays and pickup hour

In [ ]:
df_train['pickup_date'] = df_train['pickup_datetime'].dt.date
df_train['pickup_hour'] = df_train['pickup_datetime'].dt.hour
df_train['pickup_weekday'] = df_train['pickup_datetime'].dt.day_name()

holidays = [day.date() for day in calendar().holidays(start=df_train['pickup_date'].min(), end=df_train['pickup_date'].max())]
df_train['holiday'] = df_train['pickup_date'].isin(holidays)
df_train.drop('pickup_date', axis=1, inplace=True)

#### Pickup Hour

In most of cities is expected a well defined behavior regarding populational flow within the city, this way makes a sense to analyze the average trip duration per hour of the day, as can be seen in the graph bellow.

In [ ]:
avg_trip_duration_per_pickup_hour = df_train.groupby('pickup_hour')['trip_duration'].mean()
avg_trip_duration_per_pickup_hour.plot()

#### Weekday vs Pickup Hour 

As shown in the graph bellow, different day of the week present a similar behavior, similar to the previous analysis, however the duration of each trip is different for each day of the week and in weekends the curves seen to be slightly shiffted by almost 2 hours.

In [ ]:
avg_trip_duration_per_weekday = df_train.groupby(['pickup_weekday', 'pickup_hour'])['trip_duration'].mean()
avg_trip_duration_per_weekday.unstack(level=0).plot(subplots=False)

#### Holidays

Applying the same analysis for holidays we have the graph bellow, which shows the average trip duration for a normal day and for a holiday. A similar behavior can be seen for both, however the curve for holiday is slightly shifted by a little more than 2 hours

In [ ]:
avg_trip_duration_holiday = df_train.groupby(['holiday','pickup_hour'])['trip_duration'].mean()
avg_trip_duration_holiday.unstack(level=0).plot(subplots=False)
df_train.groupby(['holiday'])['trip_duration'].mean()

#### Store and Forward

The store and forward at first look may not give any information about the trip duration, but looking the graph bellow, it becomes clear there is a difference between the two. That difference may be explained by the technologies available to drivers like navigation apps, Internet or more modern cars.

In [ ]:
avg_trip_duration_per_store_flag = df_train.groupby(['store_and_fwd_flag','pickup_hour'])['trip_duration'].mean()
avg_trip_duration_per_store_flag.unstack(level=0).plot(subplots=False)
df_train.groupby(['store_and_fwd_flag'])['trip_duration'].mean()

#### Vendor ID

A naive analysis could lead to a misconception regarding the Vendor ID as a not usefull information, but given that technology has changed the private transportation market, a good routing or driver selection algorithm or papyment method can make a great difference in the trip_duration.

In [ ]:
avg_trip_duration_per_store_flag = df_train.groupby(['vendor_id','pickup_hour'])['trip_duration'].mean()
avg_trip_duration_per_store_flag.unstack(level=0).plot(subplots=False)
df_train.groupby(['vendor_id'])['trip_duration'].mean()

#### Number of Passangers

In the same way as the previous analysis, it's worth to check the behavior of the number of passagengers transported and if it might give more infomation about the trip duration.

As can be seen bellow, the graph of number of passangers shows a similar behavior for each number of passangers, but with different values for 

In [ ]:
df_train['passenger_count'].value_counts()
df_train = df_train[df_train['passenger_count']>0]

In [ ]:
# avg_trip_duration_per_passenger_count = df_train.groupby(['passenger_count'])['trip_duration'].mean()
# avg_trip_duration_per_passenger_count.plot()
avg_trip_duration_per_passenger_count = df_train[df_train['passenger_count']>0].groupby(['passenger_count','pickup_hour'])['trip_duration'].mean()
avg_trip_duration_per_passenger_count.unstack(level=0).plot(subplots=False)
df_train.groupby(['passenger_count'])['trip_duration'].mean()

## Prepare the Data to build the models

After analyzing all the field, create more features and understanding a little more about its distribution, we have to process some of the field to prepare the data, making it more suitable to the models.

### One-hot enconding

To be properly used in supervised learning models the categorical fields independent of its type, must be transformed, since wrong information can be added to the model. One classical example of that are numerical categories where there is a logic relation between the numbers (greater than, sequences, , ratios, etc.) that might not exist in the category itself.

To make the information more

In [ ]:
df_train = pd.get_dummies(df_train, columns=['vendor_id', 'passenger_count', 
                                    'store_and_fwd_flag', 'pickup_weekday', 'pickup_hour', 'holiday'])
df_train.drop(['pickup_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'], axis=1, inplace=True)
df_train.columns

Also, as could be seen in the analysis above, some field present a very skewed distribution and apply a logarithmic transformation gives a better representation of the data distribution

In [ ]:
df_train['trip_duration'] = np.log(df_train['trip_duration'] + 1)
df_train['distance'] = np.log(df_train['distance'] + 1)

## Training the Model

### Score Function

In the Kaggle competition there is no access to the true value of the trip duration for the test dataset and even though the score function is provided there is no way to calculate it. The only way to have the true score is subtmiting the values obtained when running the model to predict the trip duration for the Test dataset provided.

The score function chosen by the challenge is the [root mean square logarithmic error (RMSLE)](https://www.kaggle.com/c/nyc-taxi-trip-duration#evaluation), which is defined as follows:  

$$ \epsilon = \sqrt{ \frac{1}{n} \sum_{i=1}^{n} ( \log(p_i + 1) - \log(a_i + 1)  )^{2}    }  \;  $$

As the score function is provided we'll implement it and use the Train/Test split to evaluate our model, using it as to compare the models performance.


In [ ]:
def kaggle_score(y_true_exp, y_pred_exp):
    y_pred_exp = np.exp(y_pred) - 1
    y_true_exp = np.exp(y_true) - 1
    e_log_square = np.square( np.log(y_pred_exp + 1) - np.log(y_true_exp + 1))
    score = np.sqrt((1/len(y_true_exp)) * np.sum(e_log_square))
    return score


#### Spliting the whole train dataset using train_test_split

In [ ]:
from sklearn.model_selection import train_test_split

df_y_train = df_train['trip_duration']
df_X_train = df_train.drop(columns=['trip_duration'])

X_train, X_test, y_train, y_test = train_test_split(df_X_train,
                                                    df_y_train,
                                                    test_size = 0.3,
                                                    random_state = 3)

## Model Selection

### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
clf = model.fit(X_train, y_train)
final_score = kaggle_score( y_test, clf.predict(X_test)
                           
print('Linear Regression Score: {final_score}')

### Lasso

In [ ]:
from sklearn.linear_model import Lasso
model = Lasso(random_state=3)
clf = model.fit(X_train, y_train)
final_score = kaggle_score( y_test, clf.predict(X_test)
                           
print('Lasso Score: {final_score}')

### Ridge

In [ ]:
from sklearn.linear_model import Ridge
model = Ridge(random_state=3)
clf = model.fit(X_train, y_train)
final_score = kaggle_score( y_test, clf.predict(X_test)
                           
print('Ridge Score: {final_score}')

### ElasticNet

In [ ]:
from sklearn.linear_model import ElasticNet
model = ElasticNet(random_state=3, l1_ratio=0.0000001)
clf = model.fit(X_train, y_train)
final_score = kaggle_score( y_test, clf.predict(X_test)
                           
print('ElasticNet Score: {final_score}')

### Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(random_state=3, max_depth=None, min_samples_split=2)
clf = model.fit(X_train, y_train)
final_score = kaggle_score( y_test, clf.predict(X_test)
                           
print('Decision Tree Regressor Score: {final_score}')

### Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=3)
clf = model.fit(X_train, y_train)
final_score = kaggle_score( y_test, clf.predict(X_test)
                           
print('Random Forest Regressor Score: {final_score}')

### Gradient Boosting Regressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(random_state=3)
clf = model.fit(X_train, y_train)
final_score = kaggle_score( y_test, clf.predict(X_test)
                           
print('Gradient Boosting Regressor Score: {final_score}')

### XGBoost

In [ ]:
import xgboost as xgb
model = xgb.XGBRegressor(random_state=3)
clf = model.fit(X_train, y_train)
final_score = kaggle_score( y_test, clf.predict(X_test)
                           
print('XGBoost Score: {final_score}')

## Fine tunning the best models

**Grid Search**

**Cross Validation**

In [ ]:
all_models_final_score = {}

### Ridge

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

model = Ridge(random_state=3)
scorer = make_scorer(kaggle_score, greater_is_better= False)

parameters = {
    'alpha': [0.5, 5.0, 10.0, 50.0],
    'solver': ['auto', 'lsqr', 'sag', 'svd']
}
clf = GridSearchCV(model, param_grid= parameters, scoring= scorer, verbose= 1, n_jobs= 6, cv= 3)
grid_fit = clf.fit(X_train, y_train)

best_params_ridge = grid_fit.best_params_
best_clf_ridge = grid_fit.best_estimator_
final_score = kaggle_score(y_test, best_clf_ridge.predict(X_test))
all_models_final_score['Ridge'] = final_score

print('Ridge best parameters: {best_params}')
print('Ridge Score: {final_score}')

### Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

model = RandomForestRegressor(random_state=3)
scorer = make_scorer(kaggle_score, greater_is_better= False)

parameters = {
    'max_depth': [15, 50],
    'n_estimators': [20, 100, 200],
    'min_samples_leaf': [1, 5, 10],
    'min_samples_split': [2, 6, 10],
    'max_features': ['auto', 'sqrt']
}


clf = GridSearchCV(model, param_grid= parameters, scoring= scorer, verbose= 1, cv= 2, n_jobs=6)
grid_fit = clf.fit(X_train, y_train)

best_params_random_forest = grid_fit.best_params_
best_clf_random_forest = grid_fit.best_estimator_
final_score = kaggle_score(y_test, best_clf_random_forest.predict(X_test))
all_models_final_score['RandomForest'] = final_score

print('Gradient Boosting Regressor best parameters: {best_params}')
print('Gradient Boosting Regressor  Score: {final_score}')

### Gradient Boosting Regressor

In [1]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

model = GradientBoostingRegressor(random_state=3)
scorer = make_scorer(kaggle_score, greater_is_better= False)

parameters = {
    'max_depth': [3, 5],
    'n_estimators': [100, 200],
    'min_samples_split': [2, 6],
    'learning_rate': [0.1, 1.0]
}

clf = GridSearchCV(model, param_grid= parameters, scoring= scorer, verbose= 1, cv= 2, n_jobs=6)
grid_fit = clf.fit(X_train, y_train)

best_params_gradient_boosting = grid_fit.best_params_
best_clf_gradient_boosting = grid_fit.best_estimator_
final_score = kaggle_score(y_test, best_clf_gradient_boosting.predict(X_test))
all_models_final_score['GradientBoosting'] = final_score

print('Gradient Boosting Regressor best parameters: {best_params}')
print('Gradient Boosting Regressor  Score: {final_score}')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/luciano/Install/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-3630616f36b2>", line 1, in <module>
    from sklearn.ensemble import GradientBoostingRegressor
  File "/home/luciano/Install/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/__init__.py", line 7, in <module>
    from .forest import RandomForestClassifier
  File "/home/luciano/Install/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py", line 55, in <module>
    from ..metrics import r2_score
  File "/home/luciano/Install/anaconda3/lib/python3.6/site-packages/sklearn/metrics/__init__.py", line 7, in <module>
    from .ranking import auc
  File "/home/luciano/Install/anaconda3/lib/python3.6/site-packages/sklearn/metrics/ranking.py", line 27, in <module>
    from scipy.stats import rankdata
  File "/home/luciano/Install/anaconda3/lib/

KeyboardInterrupt: 

### XGBoost

In [2]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

model = xgb.XGBRegressor(random_state=3)
scorer = make_scorer(kaggle_score, greater_is_better=False)

parameters = {
    'max_depth': [5, 8, 10],
    'n_estimators': [200, 300],
    'learning_rate': [0.05, 0.1,],
    'reg_lambda': [1.0, 5] }

clf = GridSearchCV(model, param_grid= parameters, scoring= scorer, verbose= 1, cv=2, n_jobs=6)
grid_fit = clf.fit(X_train, y_train)

best_params_xgboost = grid_fit.best_params_
best_clf_xgboost = grid_fit.best_estimator_
final_score = kaggle_score(y_test, best_clf_xgboost.predict(X_test))
all_models_final_score['XGBoost'] = final_score

print('XGBoost best parameters: {best_params}')
print('XGBoost Score: {final_score}')

KeyboardInterrupt: 

## Predicting the Original Test Dataset and submitting to Kaggle

In [ ]:
import pandas as pd

In [ ]:
import os
BASE_PATH_KAGGLE_SUBMISISON = './out'

def create_txt_file_for_submission(df_data, file_name):
    
    final_path = os.path.normpath(os.path.join(BASE_PATH_KAGGLE_SUBMISISON,file_name))
    se_subm = pd.Series(data=df_data['trip_duration'],index=df_data.index.values)
    pd.Series.to_csv(path=file_name,
                    sep=',',
                    
                    )
    
    np.savetxt('final_submission/name.csv',se_subm, delimiter=',',
               comments='',
               newline='\n',
               fmt='%s',
               header = 'id,trip_duration')
    